In [1]:
# IMPORT LIBRARIES #
import pandas as pd
import pygsheets
import numpy as np

In [2]:
# FLAGS #

#Set to 1 to print diagnostic info
debugFlag = 0
#Set to 1 to pull info from RawNames sheet, join GPS, update, and export
updateGPSFlag = 0

In [3]:
# IMPORT DATA #

gc = pygsheets.authorize(service_file='file.json')
sh = gc.open('DashData')

if debugFlag:
    for x in sh:
        print(x)

for x in sh:
    title = x.title
    if(title=="DeliveriesOld"):
        import_sheets_OldDelDF = x.get_as_df()
    elif(title=="Deliveries"):
        import_sheets_DelDF = x.get_as_df()
    elif(title=="Days"):
        import_sheets_DaysDF = x.get_as_df()
    elif(title=="Weeks"):
        import_sheets_WeeksDF = x.get_as_df()
    elif(title=="RawNames"):
        import_sheets_RawNamesDF = x.get_as_df()
    elif(title=="GPS"):
        #for import
        import_sheets_GPSDF = x.get_as_df()
        #for export
        import_wks_gps = x
    else:
        print("Unused worksheet detected: ", x)

Unused worksheet detected:  <Worksheet 'Import' index:6>
Unused worksheet detected:  <Worksheet 'Lifetime Deliveries' index:7>
Unused worksheet detected:  <Worksheet 'Days Old' index:8>
Unused worksheet detected:  <Worksheet 'Weeks Old' index:9>


In [142]:
# Allows for disjoint cells, one for import, one for reset to import value (for ease of testing)

#DF from sheets
sheets_OldDelDF   = import_sheets_OldDelDF.copy()
sheets_DelDF      = import_sheets_DelDF.copy()
sheets_DaysDF     = import_sheets_DaysDF.copy()
sheets_WeeksDF    = import_sheets_WeeksDF.copy()
sheets_RawNamesDF = import_sheets_RawNamesDF.copy()
sheets_GPSDF      = import_sheets_GPSDF.copy()

#Sheets, ah right can't use copy on a worksheet object, might need an alternate
#wks_gps           = import_wks_gps.copy()

In [143]:
# FUNCTIONS #

# TODO: develop function to take avg distance and duration for stacked orders
def processStacks(df):
    return df

def dollars_to_floats(df, columns):
    for x in columns:
        df[x] = df[x].str.replace('$', '')
        df[x] = df[x].str.replace(',', '')
        df[x] = df[x].astype(float)
        
def hours_to_floats(df, columns):
    for x in columns:
        for index, row in df.iterrows():
            active = row[x]
            activeH = float(active[:active.index(":")])
            activeM = round(float(active[active.index(":")+1:active.index(":")+3])/60, 2)
            active = activeH + activeM
            df.at[index, x] = active
        df[x] = df[x].astype(float)

def getStartHour(df):
    df["Start Hour"] = -1
    for index, row in df.iterrows():
        startHour = row["Start Time"]
        startHour = int(startHour[:startHour.index(":")])
        df.at[index, "Start Hour"] = startHour

def addStartAndEndDate(df):
    lastRow = df.iloc[[-1]]
    #df3 = df[(df['count'] == '2') & (df['price'] == '100')]
    df = df[:-1]
    df["Start Date"] = -1
    df["End Date"]   = -1
    for index, row in df.iterrows():
        tempVar = row["Dates"]
        tempVar1 = tempVar[:tempVar.index(" ")]
        tempVar2 = tempVar[tempVar.index("- ")+2:]
        #print(tempVar1, tempVar2)
        df.at[index, "Start Date"] = tempVar1
        df.at[index, "End Date"] = tempVar2
    
    #print(df.dtypes)
    #print(lastRow.dtype)
    #print(list(df.columns.values))
    #OKAY SO ITS PROBABLY BECAUSE OF .TO_FRAME
    df = pd.concat([df, lastRow], ignore_index=True)
    df.reset_index(drop=True, inplace=True)
    #print(list(df.columns.values))
    #print(lastRow.dtype)
    #print(df.dtypes)
    return df

#Currently only valid for 2023
def date_to_datetime(df, column):
    for index, row in df.iterrows():
        df.at[index, column] = pd.Timestamp('2023/' + row[column])
    df[column] = pd.to_datetime(df[column])
    #print("TEST")
    #print(df.dtypes)
        
        
#Currently only valid for 2023
# Currently unused function
# Would we actually want this to have different dates? it's really the date the shift started
# So maybe, keep date same, but certain analysis might require that difference to be handled
def time_to_timestamp(df, DateCol, TimeCols):
    for index, row in df.iterrows():
        df.at[index, TimeCols[0]] = pd.Timestamp('2023/' + row[DateCol] + " " + row[TimeCols[0]])
        df.at[index, TimeCols[1]] = pd.Timestamp('2023/' + row[DateCol] + " " + row[TimeCols[1]])

In [144]:
# UPDATE GPS WORKSHEET #

if updateGPSFlag:
    testingDF = sheets_RawNamesDF.copy()
    testingDF = testingDF.drop_duplicates(subset=["Restaurant Name", "RAW DATA"])
    testingDF = testingDF[["Restaurant Name", "RAW DATA", "Note"]]
    newNamesDF = testingDF.merge(sheets_GPSDF, how='left')
    for index, row in newNamesDF.iterrows():
        if isinstance(row['Address'], float):
            if not row['Note'] == "":
                newNamesDF.at[index, "Address"] = row["Note"]
            else:
                print("Note not found for:", index)
        if (row['Address'] == ""):
            newNamesDF.at[index, "Resolved"] = "0"
        else:
            newNamesDF.at[index, "Resolved"] = "1"
    newNamesDF = newNamesDF.drop(columns=["Note"])
    import_wks_gps.set_dataframe(newNamesDF,(1,1))

In [145]:
# START PROCESSING #
#sheets_OldDelDF.iloc[[-1]]

In [146]:
# COMBINE DELIVERY DATA #
#sheets_frames = [sheets_OldDelDF, sheets_DelDF]
#sheets_result = pd.concat(sheets_frames)
#sheets_result = sheets_OldDelDF.append(sheets_DelDF)
#sheets_result = pd.concat([sheets_OldDelDF, sheets_DelDF], sort=False)
#sheets_result.reset_index(drop=True, inplace=True)

In [147]:
# FUNCTIONS AND OTHER SHORT PROCESSING STEPS #


#All
dollars_to_floats(sheets_OldDelDF, ["Total", "Base", "Tip", "Peak Bonus"])
#V3 only
dollars_to_floats(sheets_DelDF, ["Total", "Base", "Tip", "Peak Bonus"])
getStartHour(sheets_DelDF)
#Days
dollars_to_floats(sheets_DaysDF, ["Total", "Base", "Tip"])
hours_to_floats(sheets_DaysDF, ["Dash", "Active"])
date_to_datetime(sheets_DaysDF, "Date")
sheets_DaysDF["Day of Week"] = sheets_DaysDF["Date"].dt.day_name()
sheets_DaysDF["HourlyRate"] = sheets_DaysDF["Total"]/sheets_DaysDF["Dash"]
#Weeks
dollars_to_floats(sheets_WeeksDF, ["Total", "Base", "Tip", "Adj pay", "Other", "Pre Total"])
hours_to_floats(sheets_WeeksDF, ["Dash", "Active"])
#print(sheets_WeeksDF.dtypes)
sheets_WeeksDF = addStartAndEndDate(sheets_WeeksDF)
# pretotalactiveratio refers to weekly pre adjustment pay / time spent on deliveries
#print(sheets_WeeksDF.dtypes)
sheets_WeeksDF["pretotalactiveratio"] = round(sheets_WeeksDF["Pre Total"]/sheets_WeeksDF["Active"], 2)
sheets_WeeksDF["HourlyRate"] = sheets_WeeksDF["Total"]/sheets_WeeksDF["Dash"]




#sheets_DaysDF["HourlyRate"] = sheets_DaysDF["Total"]/sheets_DaysDF["Dash"]
#sheets_WeeksDF["HourlyRate"] = sheets_DaysDF["Total"]/sheets_DaysDF["Dash"]

/tmp/ipykernel_40047/2141852167.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[x] = df[x].str.replace('$', '')
/tmp/ipykernel_40047/2141852167.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[x] = df[x].str.replace('$', '')
/tmp/ipykernel_40047/2141852167.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[x] = df[x].str.replace('$', '')
/tmp/ipykernel_40047/2141852167.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer

In [148]:
sheets_WeeksDF

,Dates,Base,Tip,Adj pay,Other,Pre Total,Total,Active,Dash,Deliveries,Total Mileage,Active Mileage,Version,WeekID,Start Date,End Date,pretotalactiveratio,HourlyRate
0,3/27 - 4/2,108.50,88.00,40.96,0.00,196.50,237.46,6.70,10.92,22,,,1,2,3/27,4/2,29.33,21.745421
1,4/3 - 4/9,264.25,230.69,58.17,0.00,494.94,553.11,14.68,22.03,55,,,1,3,4/3,4/9,33.72,25.107127
2,4/10 - 4/16,130.75,96.13,61.07,0.00,226.88,287.95,8.63,11.17,28,,,1,4,4/10,4/16,26.29,25.778872
3,4/17 - 4/23,382.50,248.41,105.05,0.00,630.91,735.96,21.33,32.30,68,,,1,5,4/17,4/23,29.58,22.785139
4,4/24 - 4/30,430.00,299.73,138.76,2.62,729.73,871.11,25.50,41.17,84,,,1,6,4/24,4/30,28.62,21.158854
5,5/1 - 5/7,618.50,445.32,121.74,0.00,1063.82,1185.56,32.40,50.67,114,,,1,7,5/1,5/7,32.83,23.397671
6,5/8 - 5/14,758.25,567.84,155.48,6.25,1326.09,1487.82,40.58,58.13,138,,,2,8,5/8,5/14,32.68,25.594702
7,5/15 - 5/21,303.75,239.25,99.27,0.00,543.00,642.27,17.53,25.40,64,,,2,9,5/15,5/21,30.98,25.286220
8,5/22 - 5/28,205.50,189.19,63.63,0.00,394.69,458.32,11.65,16.67,38,230,198.36,3,10,5/22,5/28,33.88,27.493701
9,5/29 - 6/4,342.75,309.88,115.30,0.00,652.63,767.93,19.40,26.53,63,389,333.5,3,11,5/29,6/4,33.64,28.945722


In [149]:
# BEGIN ADJ PAY SECTION #

# This establishes the estimated pay adjustment estimates by delivery, date, and week
# This is calculated using time spent delivering and distance approximations
# The real adjusted pay is based on weekly values
# I suspect the primary cause for difference between estimate and real values stems from 
#how I estimate mileage vs how the app tracks mileage

# DFs are copied to avoid excluding dropped values in later analysis



#if debugFlag:
#    sheets_DelDF.tail()

In [150]:
#Per delivery estimations
#sheets_DelDF1 = sheets_DelDF.copy()

#Convert to float if possible, else convert to NaN
#sheets_DelDF1["Duration"] = pd.to_numeric(sheets_DelDF1["Duration"], errors='coerce')
sheets_DelDF["Duration"] = pd.to_numeric(sheets_DelDF["Duration"])
#Drop NaN rows
#sheets_DelDF1 = sheets_DelDF1[sheets_DelDF1["Duration"].notna()]
#Calculate estimate adjustment per delivery
sheets_DelDF["Est Adj"] = (sheets_DelDF["Duration"]/60*18
                           + sheets_DelDF["Distance"]*(0.34) ) - sheets_DelDF["Base"]
#Duration is over 60 because its value is in minutes, values from day and week are in hours

#if debugFlag:
#    sheets_DelDF.tail()

In [151]:
#if debugFlag:
#    sheets_DaysDF.tail()

In [152]:
#Per day estimations
sheets_DaysDF1 = sheets_DaysDF.copy()
sheets_DaysDF1 = sheets_DaysDF1[sheets_DaysDF1["Version"] == 3]
sheets_DaysDF1["Est Adj"] = (sheets_DaysDF1["Active"]*(18) 
                            + sheets_DaysDF1["Active Mileage"]*(0.34) ) - sheets_DaysDF1["Base"]

In [153]:
#if debugFlag:
#    sheets_DaysDFv3.tail()

In [154]:
#Per week estimations
sheets_WeeksDF1 = sheets_WeeksDF.copy()
sheets_WeeksDF1 = sheets_WeeksDF1[sheets_WeeksDF1["Version"] == 3] #totals doesn't have v3 so its dropped
sheets_WeeksDF1["Est Adj"] = (sheets_WeeksDF1["Active"]*(18) 
                            + sheets_WeeksDF1["Active Mileage"]*(0.34) ) - sheets_WeeksDF1["Base"]

In [155]:
#appends


# Sample values
if debugFlag:
    print("Del DF Sample\n")
    print(sheets_DelDF.iloc[-1])
    print("\nDays DF Sample\n")
    print(sheets_DaysDF1.iloc[-1])
    print("\nWeeks DF Sample\n")
    print(sheets_WeeksDF1.iloc[-1])
    
    
# END ADJ PAY SECTION #

In [156]:
print(list(sheets_DelDF.columns.values))
print()
print(list(sheets_DaysDF.columns.values))
print()
print(list(sheets_WeeksDF.columns.values))
print()
print(sheets_DelDF["Start Hour"].unique())

['ID', 'Date', 'Restaurant Name', 'Total', 'Base', 'Tip', 'Peak Bonus', 'Stacked', 'Start Time', 'Distance', 'End Time', 'Duration', 'Version', 'Start Hour', 'Est Adj']

['Date', 'Total', 'Base', 'Tip', 'Start (24)', 'End (24)', 'Active', 'Dash', 'Offers', 'Deliveries', 'Mileage Start', 'Mileage End', 'Total Mileage', 'Active Mileage', 'Version', 'Pauses', 'WeekID', 'Day of Week', 'HourlyRate']

['Dates', 'Base', 'Tip', 'Adj pay', 'Other', 'Pre Total', 'Total', 'Active', 'Dash', 'Deliveries', 'Total Mileage', 'Active Mileage', 'Version', 'WeekID', 'Start Date', 'End Date', 'pretotalactiveratio', 'HourlyRate']

[ 2  1  5 12  7  8  4  6  3  9 10 11]


In [157]:
sheets_WeeksDF.iloc[-1]

Dates                    TOTALS:
Base                     5591.25
Tip                      4399.11
Adj pay                  1652.51
Other                      32.75
Pre Total                9990.36
Total                   11675.62
Active                    317.83
Dash                      457.08
Deliveries                  1068
Total Mileage               2718
Active Mileage           2621.16
Version                        -
WeekID                          
Start Date                   NaN
End Date                     NaN
pretotalactiveratio        31.43
HourlyRate             25.543931
Name: 17, dtype: object

In [158]:
#ACTUALLY LETS DO THE RESULTS JOIN HERE

sheets_result = pd.concat([sheets_OldDelDF, sheets_DelDF], sort=False)
sheets_result.reset_index(drop=True, inplace=True)

#NO okay so we have to concat df1 (v3) to df (v1,2), THIS SHOULD WORK
sheets_DaysDF0 = sheets_DaysDF[sheets_DaysDF["Version"] != 3]
sheets_WeeksDF0 = sheets_WeeksDF[sheets_WeeksDF["Version"] != 3] #THIS PUTS TOTALS IN THE MIDDLE
total_S = sheets_WeeksDF0.iloc[[-1]]
sheets_WeeksDF0 = sheets_WeeksDF0[:-1]
#okay so what if we grab wksdf0.iloc-1, store, drop, then add it at the end

sheets_DaysDF = pd.concat([sheets_DaysDF0, sheets_DaysDF1], sort=False)
sheets_DaysDF.reset_index(drop=True, inplace=True)

sheets_WeeksDF = pd.concat([sheets_WeeksDF0, sheets_WeeksDF1], sort=False)
sheets_WeeksDF = pd.concat([sheets_WeeksDF, total_S], ignore_index=True)
sheets_WeeksDF.reset_index(drop=True, inplace=True)
#CHECK DTYPES OF sheets_WeeksDF


In [159]:
sheets_DaysDF.dtypes

Date              datetime64[ns]
Total                    float64
Base                     float64
Tip                      float64
Start (24)                object
End (24)                  object
Active                   float64
Dash                     float64
Offers                     int64
Deliveries                 int64
Mileage Start             object
Mileage End               object
Total Mileage             object
Active Mileage            object
Version                    int64
Pauses                    object
WeekID                     int64
Day of Week               object
HourlyRate               float64
Est Adj                   object
dtype: object

In [160]:
# END PROCESSING #
# START INITIAL EXPLORATION #

In [161]:
#Percentage of deliveries taken that tip
numUntipped = (sheets_result["Tip"] == 0.0).sum()
numTipped = (sheets_result["Tip"] != 0.0).sum()
percTipped = round((numTipped/(numUntipped+numTipped)), 4)
print(percTipped*100, "% of accepted deliveries included tip")

#Average tip, including and excluding untipped deliveries
TipAvg = round(sheets_result["Tip"].mean(), 2)
print("$", TipAvg, "average tip value including orders with no tip")
TipAvg2 = round(sheets_result[sheets_result["Tip"] !=0]["Tip"].mean(), 2)
print("$", TipAvg2, "average tip value excluding orders with no tip")

# Overall hourly rate
hourlyRate = round(sheets_WeeksDF.iloc[-1]["Total"]/sheets_WeeksDF.iloc[-1]["Dash"], 2)
print("Overall hourly pay rate is $", hourlyRate)

# Max and Average values by Delivery, Day, and Week
# Some of these values are wrong, because I have not yet implemented a stack value handler
print()
print("Daily Max Values")
print(sheets_DaysDF[["Total", "Base", "Tip", "Active", "Dash", "Deliveries"]].max())
print(sheets_DaysDF[sheets_DaysDF["Version"] == 3][["Total Mileage", "Active Mileage"]].max(), "\n" )
print("Daily Average Values")
print(sheets_DaysDF[["Total", "Base", "Tip", "Active", "Dash", "Deliveries"]].mean())
print(sheets_DaysDF[sheets_DaysDF["Version"] == 3][["Total Mileage", "Active Mileage"]].mean(), "\n" )
print()
# Some of these values are wrong, because I have not yet implemented a stack value handler
print("Weekly Max Values")
#
print(sheets_WeeksDF[["Base", "Tip", "Adj pay", "Pre Total", "Total", "Active", "Dash", "Deliveries"]].max())
#
print(sheets_WeeksDF[sheets_WeeksDF["Version"] == 3][["Total Mileage", "Active Mileage"]].max(), "\n")
print("Weekly Average Values")
#
print(sheets_WeeksDF[["Base", "Tip", "Adj pay", "Pre Total", "Total", "Active", "Dash", "Deliveries"]].mean())
#
print(sheets_WeeksDF[sheets_WeeksDF["Version"] == 3][["Total Mileage", "Active Mileage"]].mean(), "\n")
print()
# Some of these values are wrong, because I have not yet implemented a stack value handler
print("Delivery Max Values")
#
print(sheets_DelDF[["Distance", "Duration"]].max())
print(sheets_result[["Total"]].max(), "\n")
print("Delivery Average Values")
#
print(sheets_DelDF[["Distance", "Duration"]].mean())
print(sheets_result[["Total"]].mean(), "\n")

# END INITIAL EXPLORATION #

90.74 % of accepted deliveries included tip
$ 4.12 average tip value including orders with no tip
$ 4.54 average tip value excluding orders with no tip
Overall hourly pay rate is $ 25.54

Daily Max Values
Total         333.57
Base          191.50
Tip           149.53
Active          9.67
Dash           13.60
Deliveries     33.00
dtype: float64
Total Mileage       151
Active Mileage    229.2
dtype: object 

Daily Average Values
Total         163.776393
Base           91.659836
Tip            72.116557
Active          5.198525
Dash            7.483115
Deliveries     17.508197
dtype: float64
Total Mileage     97.071429
Active Mileage    93.614286
dtype: float64 


Weekly Max Values
Base           5591.25
Tip            4399.11
Adj pay        1652.51
Pre Total      9990.36
Total         11675.62
Active          317.83
Dash            457.08
Deliveries     1068.00
dtype: float64
Total Mileage       633
Active Mileage    619.4
dtype: object 

Weekly Average Values
Base           621.250000
T

In [162]:
# Restaurants rankable by avg or mean base/tip/total
# Yes, total is almost always base+tip, but total INCLUDES peak pay, which may be desirable information

tempDF = sheets_result.copy()
#The line below drops restaurants that only appear once, we may no longer want this to happen
#Commented out for now, may make sense to use when considering outliers for averages
#tempDF = tempDF[sheets_result["Restaurant Name"].duplicated(keep=False) == True]
tempDF = tempDF.groupby(["Restaurant Name"]).agg({"Base": [np.mean, np.sum], 
                                                  "Tip": [np.mean, np.sum],
                                                  "Total": [np.mean, np.sum],
                                                 'Restaurant Name': 'size'})
tempDF.columns = ['Avg Base', 'Sum Base', 'Avg Tip', 
                  'Sum Tip', 'Avg Total', 'Sum Total', "Deliveries"]

tempDF = tempDF.round({'Avg Base': 2, 
                       'Sum Base': 2,
                       'Avg Tip': 2,
                       'Sum Tip': 2,
                       'Avg Total': 2,
                       'Sum Total': 2})

#Change sort value to see differences
tempDF1 = tempDF.index.copy()
restaurantsDF = tempDF.copy()
tempDF.sort_values("Sum Total", ascending=False)







#RESTAURANTS DF

,Avg Base,Sum Base,Avg Tip,Sum Tip,Avg Total,Sum Total,Deliveries
Restaurant Name,,,,,,,
Chick-fil-A,5.70,689.75,3.61,436.39,9.31,1126.14,121
Taco Bell,4.79,315.87,3.44,226.75,8.70,574.12,66
McDonald's,4.59,316.50,3.30,227.42,7.90,544.92,69
Dave's Hot Chicken,5.24,256.75,4.36,213.50,9.60,470.25,49
Chipotle,4.69,248.50,3.52,186.42,8.24,436.92,53
...,...,...,...,...,...,...,...
Everbowl,4.00,4.00,2.00,2.00,6.00,6.00,1
Kung Fu Noodle,5.50,5.50,0.00,0.00,5.50,5.50,1
Papa Johns,4.00,4.00,1.00,1.00,5.00,5.00,1


In [163]:
#Dollars Earned vs Miles Driven is another factor I wanted to consider in the overall earnings equation

# Grab v3 deliveries, drop rows without known distance values
# Group by restaurant name, sum on total and distance
# Evaluate total pay over total distance as new column
# Sort low to high



#WE COULD GRAB RESULTS.VERSION==3 INSTEAD 
tempDF = sheets_result[sheets_result["Version"] == 3].copy()
#tempDF = sheets_DelDF.copy()
#tempDF = tempDF[tempDF.Distance!="-"]
#tempDF["Distance"] = tempDF["Distance"].astype(float)
tempDF = tempDF.groupby(["Restaurant Name"]).agg({"Total": "sum", "Distance": "sum"})
tempDF = tempDF.round({'Base': 2, 'Tip': 2})
tempDF["DollarMilesRatio"] = tempDF["Total"]/tempDF["Distance"]
#tempDF.sort_values(by="DollarMilesRatio")
tempDF2 = tempDF.index.copy()

restaurantsDF = restaurantsDF.join(tempDF["DollarMilesRatio"])

tempDF


#JOIN WITH RESTAURANTS DF


#160 in total, 115 in v3? so 45 restaurants unique to v1 and 2??

#Can I select results where version !=3, get rest count name no
#can we select rest from temp1 not in temp2?

,Total,Distance,DollarMilesRatio
Restaurant Name,,,
7-Eleven,122.37,92.4,1.324351
Advanced Auto Parts,13.75,4.6,2.989130
Albertsons,34.17,8.8,3.882955
Angry Chickz,11.25,7.9,1.424051
Anita's Mexican Restaurant & Cantina,28.25,16.9,1.671598
...,...,...,...
Wendy's,45.25,17.8,2.542135
Wings-N-Things,14.75,7.9,1.867089
Wingstop,27.50,17.9,1.536313


In [164]:
restaurantsDF

,Avg Base,Sum Base,Avg Tip,Sum Tip,Avg Total,Sum Total,Deliveries,DollarMilesRatio
Restaurant Name,,,,,,,,
7-Eleven,3.91,132.87,2.44,83.00,6.50,220.87,34,1.324351
Acai Superfruits,5.25,5.25,5.00,5.00,10.25,10.25,1,NaN
Advanced Auto Parts,5.56,22.25,0.00,0.00,5.56,22.25,4,2.989130
Albertsons,4.58,13.75,12.39,37.17,17.64,52.92,3,3.882955
Angry Chickz,6.25,6.25,5.00,5.00,11.25,11.25,1,1.424051
...,...,...,...,...,...,...,...,...
Wild Burger,5.00,5.00,4.00,4.00,9.00,9.00,1,NaN
Wings-N-Things,4.56,18.25,3.62,14.50,8.19,32.75,4,1.867089
Wingstop,5.33,16.00,3.83,11.50,9.17,27.50,3,1.536313


In [165]:
tempDF3 = tempDF1[~tempDF1.isin(tempDF2)].dropna(how = 'all')
tempDF3 #These are the restaurants from just v1, v2
#Thought there were too many but, there were actually a lot that haven't been seen since v3

Index(['Acai Superfruits', 'Beach Hut Deli', 'Bloom Haus',
       'Brooklyn Boyz Pizza', 'Chili's Grill & Bar', 'Chin's Szechwan',
       'Cream Of The Crop', 'Epic Wings', 'Erika's Mexican Food & Seafood',
       'Everbowl', 'FTD', 'Firehouse Subs', 'Greek Corner Cafe',
       'Guanatos City Mexican Restaurant', 'Handel's Homemade Ice Cream',
       'Harney Sushi', 'Jamba', 'Joe's Crab Shack', 'Krispy Kreme',
       'Mikko Sushi', 'North Park Produce Bakery and Grill',
       'Ocean Thai Cuisine', 'PETCO', 'Pit Stop Diner', 'Pollos Maria',
       'Quik Wok', 'Sizzler', 'Sonic Drive-In', 'Super Mega Dilla',
       'TZONE Oceanside', 'Tapioca Express', 'TenderFix by Noah Schnapp',
       'The Cheesecake Factory', 'The Cheesesteak Grill', 'The Meltdown',
       'The Poke Cafe', 'Tony's Fresh Mexican Food',
       'Umi Japanese Grill & Sushi', 'Village Kabob', 'Wienerschnitzel',
       'Wild Burger'],
      dtype='object', name='Restaurant Name')

In [166]:
# Average $/hour and tip amount by day of week
# NOTE: This excludes pay adjustments

tempDF = sheets_DaysDF.copy()

# Can move this up to formatting function calls
# Also, check if necessary to convert from string to timestamp to datetime
#date_to_timestamp(tempDF, "Date")
#tempDF["Date"] = pd.to_datetime(tempDF["Date"])

#Adds column to DaysDF for the day of the week (Monday, Thursday, etc.)
#tempDF["Day of Week"] = tempDF["Date"].dt.day_name()

tempDF = tempDF.groupby(["Day of Week"]).agg(
    {"Dash": "sum", 
     "Total": [np.mean, np.sum], 
     "Tip": "sum", 
     "Deliveries": "sum"})
tempDF.columns = ['Sum Dash', 'Avg Daily Total', 'Sum Total', 'Sum Tip', 'Sum Deliveries']

tempDF["$/hr"] = tempDF["Sum Total"]/tempDF["Sum Dash"]
tempDF["Avg Delivery Tip"] = tempDF["Sum Tip"]/tempDF["Sum Deliveries"]
tempDF["Avg Delivery Total"] = tempDF["Sum Total"]/tempDF["Sum Deliveries"]

tempDF = tempDF.round({'Sum Dash': 2, 
                       'Avg Total': 2, 
                       'Sum Total':2, 
                       'Sum Tip': 2,
                       '$/hr': 2,
                       'AvgTip': 2,
                       'AvgTotal':2})

display(tempDF)

,Sum Dash,Avg Daily Total,Sum Total,Sum Tip,Sum Deliveries,$/hr,Avg Delivery Tip,Avg Delivery Total
Day of Week,,,,,,,,
Friday,135.57,189.533125,3032.53,1385.78,316,22.37,4.385380,9.596614
Monday,17.85,125.823333,377.47,158.97,41,21.15,3.877317,9.206585
Saturday,112.83,185.460714,2596.45,1144.95,270,23.01,4.240556,9.616481
Sunday,49.58,147.347500,1178.78,545.53,122,23.78,4.471557,9.662131
Thursday,72.83,140.773636,1548.51,656.01,176,21.26,3.727330,8.798352
Wednesday,67.81,139.624444,1256.62,507.87,143,18.53,3.551538,8.787552


In [167]:
sheets_WeeksDF.tail()

,Dates,Base,Tip,Adj pay,Other,Pre Total,Total,Active,Dash,Deliveries,Total Mileage,Active Mileage,Version,WeekID,Start Date,End Date,pretotalactiveratio,HourlyRate,Est Adj
13,7/3 - 7/9,114.25,105.10,54.99,7.75,219.35,282.09,7.83,10.17,23,108,98.1,3,15,7/3,7/9,28.01,27.737463,60.044
14,7/17 - 7/23,454.75,425.11,151.31,3.38,879.86,1034.55,26.22,37.30,91,471,438.6,3,16,7/17,7/23,33.56,27.735925,166.334
15,7/24 - 7/30,278.25,209.79,71.96,0.00,488.04,560.00,14.82,19.52,50,285,271.8,3,17,7/24,7/30,32.93,28.688525,80.922
16,7/31 - 8/6,248.75,183.44,74.75,0.00,432.19,506.94,13.75,18.57,44,272,386.2,3,18,7/31,8/6,31.43,27.298869,130.058
17,TOTALS:,5591.25,4399.11,1652.51,32.75,9990.36,11675.62,317.83,457.08,1068,2718,2621.16,-,,NaN,NaN,31.43,25.543931,NaN


In [168]:
sheets_DaysDF.tail()

,Date,Total,Base,Tip,Start (24),End (24),Active,Dash,Offers,Deliveries,Mileage Start,Mileage End,Total Mileage,Active Mileage,Version,Pauses,WeekID,Day of Week,HourlyRate,Est Adj
56,2023-07-28,162.24,89.25,72.99,15:06,21:41,4.62,6.57,14,15,70962,71071,109,90.1,3,"(6:27pm, 6:57pm)",17,Friday,24.694064,24.544
57,2023-07-29,111.25,66.75,44.50,18:00,20:10,3.55,4.15,10,12,71086,71149,63,84.3,3,0,17,Saturday,26.807229,25.812
58,2023-08-04,163.44,96.00,67.44,13:49,21:24,5.38,7.58,11,15,71226,71340,114,229.2,3,0,18,Friday,21.562005,78.768
59,2023-08-05,173.75,96.25,77.50,15:42,22:27,5.03,6.75,17,18,71345,71447,102,106.3,3,"(6:28pm, 6:52pm)",18,Saturday,25.740741,30.432
60,2023-08-06,95.00,56.50,38.50,16:37,20:50,3.32,4.22,11,11,71454,71510,56,50.7,3,"(7:04pm, 7:09pm)",18,Sunday,22.511848,20.498
